In [1]:
from pycaret.classification import *
from pycaret.classification import load_experiment

In [2]:
import pandas as pd
import numpy as np
import pickle

from tqdm import tqdm
tqdm.pandas()

import os
from pprint import pprint
import re
from collections import defaultdict
import copy
from sklearn.model_selection import train_test_split

from util import *
import evaluate_auth



In [3]:
from datetime import datetime

time_begin = datetime.now()
print("Begining Time:\n", time_begin)

Begining Time:
 2024-02-26 18:52:07.350917


In [4]:
DATA_ADDRESS = "./data"
# os.listdir(DATA_ADDRESS)
# TRAIN_DIR = os.path.join(DATA_ADDRESS,'preprocessed','train')
TEST_DIR = os.path.join(DATA_ADDRESS,'preprocessed','test')

# os.listdir(TRAIN_DIR)

In [5]:
data_test = load_data(
    dir_feature = TEST_DIR,
    file_prefix = 'mfcc_len5_fft2048_mels128_mfcc17_',
    dir_df_index = os.path.join(DATA_ADDRESS,'df_index_test.pkl'),
    n_interval=500,
    flatten=True)

100%|██████████| 50/50 [00:00<00:00, 195.58it/s]


Loaded dataframe
 with flattend X
 with shape: (24800, 3673)


In [6]:
data_test

,0,1,2,3,4,5,6,7,8,9,...,3663,3664,3665,3666,3667,3668,3669,3670,3671,target
0,-77.335411,-76.044968,-75.780907,-76.207321,-76.968941,-78.054825,-78.889557,-79.136742,-78.707405,-77.364380,...,1.666882,1.499124,1.059202,0.374428,-0.468886,-1.169286,-0.854867,-0.729526,-0.158081,192.0
1,-73.302490,-74.613892,-75.964531,-74.992409,-73.675034,-73.362015,-74.261330,-75.696556,-76.656960,-77.399261,...,0.682681,0.048326,-0.012902,0.545132,1.274721,1.936445,2.379305,1.697993,1.388240,192.0
2,-70.028969,-70.884056,-71.527603,-70.559921,-69.526054,-68.636116,-67.744370,-66.134888,-65.494141,-67.076927,...,-0.003192,0.087003,0.376900,-0.297952,-0.694625,-0.153152,-0.107342,-0.534249,-0.441920,192.0
3,-69.520584,-70.708282,-73.370262,-74.992615,-76.616226,-77.995964,-78.455650,-79.094627,-80.521629,-80.936478,...,0.023778,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,192.0
4,-71.714066,-68.675499,-67.467400,-66.962158,-67.511063,-67.628944,-67.616135,-67.188828,-67.699387,-67.895012,...,1.495647,0.805673,0.509418,0.759485,0.382243,0.110432,-0.015470,-0.084720,0.296677,192.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24795,-74.823402,-75.183197,-72.505684,-62.229679,-56.033108,-53.905876,-54.594402,-55.899620,-57.826359,-59.926170,...,0.023778,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,2137.0
24796,-81.253174,-79.992989,-79.081581,-78.866364,-79.583725,-79.730759,-80.617554,-81.187675,-81.978432,-81.488297,...,0.290756,0.395896,0.083245,-1.099505,-1.459450,-1.171437,-1.480871,-1.380765,-0.541722,2137.0
24797,-60.312725,-66.106125,-78.295441,-81.621445,-83.470802,-83.794319,-83.916412,-83.942101,-83.948776,-83.949234,...,0.042555,0.023779,0.036162,0.054693,0.106021,0.065853,0.037056,0.052584,0.061042,2137.0
24798,-84.050369,-84.043938,-84.046204,-84.046669,-84.046188,-84.045700,-83.349434,-79.142738,-68.420471,-62.473373,...,0.042842,-0.031567,-0.049336,-0.035447,0.158722,0.485223,0.065013,-0.336305,-0.680135,2137.0


In [7]:
# logistic regression
clf_pre = load_model("model/test_source/logesticRegression")


Transformation Pipeline and Model Successfully Loaded


In [8]:
clf_pre

Pipeline(memory=Memory(location=None),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(include=['0', '1', '2', '3', '4', '5', '6',
                                             '7', '8', '9', '10', '11', '12',
                                             '13', '14', '15', '16', '17', '18',
                                             '19', '20', '21', '22', '23', '24',
                                             '25', '26', '27', '28', '29', ...],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=[],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('trained_model',
                 LogisticRegression(max_iter=1000, random_state=42))])

1. For each unique speaker's id in column `target`
   1. Change the `target` = the current iterating speaker to 1 and the rest to 0 (make it binary classification)
   2. Split the speech into training and testing sets.
   3. Fine-tune the pretrained model `clf = copy.deepcopy(clf_pre)`
   4. Accept the authentication if the model have at least 85 of confidence (maybe)
   5. Evaluate the Authentication performance with EER, FRR, FAR, accuracy, recall, precision, f1. Use a dictionary to store the evaluation, use the original speaker_id as key
2. Evaluate the average evaluation scores for all speakers on both train and test
3. Print the evaluation into a pandas dataframe

In [9]:
# V1.1

dict_evaluation = defaultdict(dict)
# Step 1
unique_speakers = data_test['target'].unique()

for speaker_id in tqdm(unique_speakers):
    data_binary = data_test.copy()
    data_binary['target_binary'] = np.where(data_binary['target'] == speaker_id, 1, 0)
    
    # Step 2
    X = data_binary.drop(['target', 'target_binary'], axis=1)
    y = data_binary['target_binary']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)
    
    # Step 3
    clf = copy.deepcopy(clf_pre)
    clf.fit(X_train, y_train)
    
    # Step 4 - Evaluation
    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)

    dict_evaluation['train'][speaker_id] = evaluate_auth.evaluate_model(y_train,y_pred_train)
    dict_evaluation['test'][speaker_id] = evaluate_auth.evaluate_model(y_test,y_pred_test)
    



100%|██████████| 100/100 [1:01:14<00:00, 36.74s/it]


In [10]:
dict_evaluation

defaultdict(dict,
            {'train': {192.0: {'eer': 0.0,
               'frr': 0.0,
               'far': 0.0,
               'acc': 1.0,
               'recall': 1.0,
               'precision': 1.0,
               'f1': 1.0},
              5333.0: {'eer': 0.0,
               'frr': 0.0,
               'far': 0.0,
               'acc': 1.0,
               'recall': 1.0,
               'precision': 1.0,
               'f1': 1.0},
              606.0: {'eer': 0.0,
               'frr': 0.0,
               'far': 0.0,
               'acc': 1.0,
               'recall': 1.0,
               'precision': 1.0,
               'f1': 1.0},
              1079.0: {'eer': 0.0,
               'frr': 0.0,
               'far': 0.0,
               'acc': 1.0,
               'recall': 1.0,
               'precision': 1.0,
               'f1': 1.0},
              5029.0: {'eer': 0.0,
               'frr': 0.0,
               'far': 0.0,
               'acc': 1.0,
               'recall': 1.0,
     

In [11]:
evaluate_auth.evaluation_to_df(dict_evaluation)


,eer,frr,far,acc,recall,precision,f1
train,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
test,0.052346,0.056428,0.000134,0.999302,0.943572,0.986429,0.964202


In [12]:
time_end = datetime.now()
print("End Time:\n", time_end)
print("Run Time:\n", time_end-time_begin)

End Time:
 2024-02-26 19:53:23.604063
Run Time:
 1:01:16.253146


first out put

source no copy

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>eer</th>
      <th>frr</th>
      <th>acc</th>
      <th>recall</th>
      <th>precision</th>
      <th>f1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>train</th>
      <td>0.005000</td>
      <td>0.01</td>
      <td>0.999952</td>
      <td>0.99</td>
      <td>0.99</td>
      <td>0.99</td>
    </tr>
    <tr>
      <th>test</th>
      <td>0.400066</td>
      <td>NaN</td>
      <td>0.989742</td>
      <td>0.03</td>
      <td>0.03</td>
      <td>0.03</td>
    </tr>
  </tbody>
</table>
</div>

Second out put

all data

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>eer</th>
      <th>frr</th>
      <th>acc</th>
      <th>recall</th>
      <th>precision</th>
      <th>f1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>train</th>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>1.000000</td>
      <td>1.000000</td>
      <td>1.000000</td>
      <td>1.000000</td>
    </tr>
    <tr>
      <th>test</th>
      <td>0.053402</td>
      <td>0.058007</td>
      <td>0.999317</td>
      <td>0.941993</td>
      <td>0.989472</td>
      <td>0.964707</td>
    </tr>
  </tbody>
</table>
</div>

logistic regression

output 3

source stratified split

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>eer</th>
      <th>frr</th>
      <th>far</th>
      <th>acc</th>
      <th>recall</th>
      <th>precision</th>
      <th>f1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>train</th>
      <td>0.00500</td>
      <td>0.01</td>
      <td>0.000000</td>
      <td>0.999952</td>
      <td>0.99</td>
      <td>0.990000</td>
      <td>0.990000</td>
    </tr>
    <tr>
      <th>test</th>
      <td>0.49204</td>
      <td>0.98</td>
      <td>0.001501</td>
      <td>0.987032</td>
      <td>0.02</td>
      <td>0.023333</td>
      <td>0.020667</td>
    </tr>
  </tbody>
</table>
</div>

output 4

all data stratified split

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>eer</th>
      <th>frr</th>
      <th>far</th>
      <th>acc</th>
      <th>recall</th>
      <th>precision</th>
      <th>f1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>train</th>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>1.000000</td>
      <td>1.000000</td>
      <td>1.000000</td>
      <td>1.000000</td>
    </tr>
    <tr>
      <th>test</th>
      <td>0.052346</td>
      <td>0.056428</td>
      <td>0.000134</td>
      <td>0.999302</td>
      <td>0.943572</td>
      <td>0.986429</td>
      <td>0.964202</td>
    </tr>
  </tbody>
</table>
</div>